In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import meterdatalogic as ml

In [ ]:
df = ml.ingest.from_nem12("./data/QB06066551_20250101_20251031_20251107130416_ENERGEXP_DETAILED.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(df)
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]
df

In [ ]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
monthly

In [ ]:
## My current plan 
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand(
            name="all_times",     # must match the column name produced by transform.tou_bins
            start="00:00",
            end="24:00",          # full-day window
            rate_c_per_kwh=43.25  # $0.4325/kWh -> 43.25 cents
        )
    ],
    fixed_c_per_day=170.56,        # $1.7056/day -> 170.56 cents
    feed_in_c_per_kwh=60.0,        # $0.60/kWh -> 60.0 cents
    demand=None                    # no demand charge on this plan
)


In [ ]:
cost = ml.pricing.estimate_monthly_cost(df, plan)  # month, components, total
cost

In [ ]:
cycles = [
    ("2025-03-01", "2025-03-30"),
    ("2025-03-31", "2025-04-30"),
    ("2025-05-01", "2025-05-30"),
    ("2025-05-31", "2025-06-30"),
    ("2025-07-01", "2025-07-30"),
]

bills = ml.pricing.estimate_cycle_costs(
    df, plan, cycles,
    pay_on_time_discount=0.07,   
    include_gst=True            
)
# bills["total"] = bills["total"].round(2)
bills